## Setup

In [1]:
import os
import requests
import json
from pathlib import Path

import sys
sys.path.append('../')
from modules.helpers import get_access_token, load_eoepca_state, test_cell, test_results

Load `eoepca state` environment

In [2]:
load_eoepca_state()

In [3]:
platform_domain = os.environ.get("INGRESS_HOST")
resource_registration_domain = f'{os.environ.get("HTTP_SCHEME")}://registration-api.{platform_domain}'

print(f"Resource Registration API: {resource_registration_domain}")

Resource Registration API: https://registration-api.notebook-test.develop.eoepca.org


## Validate Resource Registration API Endpoints

In [4]:
endpoints = [
    ("Landing Page", resource_registration_domain),
    ("Swagger UI", f"{resource_registration_domain}/openapi?f=html"),
    ("Conformance Declaration", f"{resource_registration_domain}/conformance"),
    ("Processes List", f"{resource_registration_domain}/processes"),
    ("Jobs Endpoint", f"{resource_registration_domain}/jobs")
]

for name, url in endpoints:
    response = requests.get(url)
    print(f"{name} ({url}): {response.status_code}")

Landing Page (https://registration-api.notebook-test.develop.eoepca.org): 200
Swagger UI (https://registration-api.notebook-test.develop.eoepca.org/openapi?f=html): 200
Conformance Declaration (https://registration-api.notebook-test.develop.eoepca.org/conformance): 200
Processes List (https://registration-api.notebook-test.develop.eoepca.org/processes): 200
Jobs Endpoint (https://registration-api.notebook-test.develop.eoepca.org/jobs): 200


## Inspect Available Processes

In [5]:
processes_url = f"{resource_registration_domain}/processes"
response = requests.get(processes_url)
print(json.dumps(response.json(), indent=2)[:500])

{
  "processes": [
    {
      "version": "0.1.0",
      "id": "deregister",
      "title": "Resource deregistration",
      "description": "Resource deregistration",
      "jobControlOptions": [
        "sync-execute",
        "async-execute"
      ],
      "keywords": [
        "resource",
        "deregistration"
      ],
      "links": [
        {
          "type": "text/html",
          "rel": "about",
          "title": "information",
          "href": "https://eoepca.readthedocs.io/projec


## Registering Remote STAC Collection

In [11]:
register_url = f"{resource_registration_domain}/processes/register/execution"

payload = {
    "inputs": {
        "type": "collection",
        "source": "https://raw.githubusercontent.com/radiantearth/stac-spec/master/examples/collection.json",
        "target": f"https://resource-catalogue.{platform_domain}/stac"
    }
}
response = requests.post(register_url, json=payload)
registration_result = response.json()
print(registration_result)

{'id': 'registrar', 'resource-and-data-catalogue-link': {'href': 'https://resource-catalogue.notebook-test.develop.eoepca.org/stac/collections/metadata:main/items/simple-collection', 'rel': 'item', 'type': 'application/geo+json'}}


## Deregistering a STAC Collection

In [24]:
deregister_url = f"{resource_registration_domain}/processes/deregister/execution"
payload = {
    "inputs": {
        "id": "simple-collection",
        "target": {
            "href": f"https://resource-catalogue.{platform_domain}",
            "rel": "https://api.stacspec.org/v1.0.0/core"
        }
    }
}
response = requests.post(deregister_url, json=payload)
print(response.json())

{'type': 'InvalidParameterValue', 'code': 'InvalidParameterValue', 'description': 'Error executing process: ["$.target: {\'href\': \'https://resource-catalogue.notebook-test.develop.eoepca.org\', \'rel\': \'https://api.stacspec.org/v1.0.0/core\'} is not of type \'string\'"]'}


## Registering a Sample STAC Item

In [13]:
stac_item_url = "https://raw.githubusercontent.com/EOEPCA/deployment-guide/main/scripts/resource-registration/data/simple-item.json"

stac_item_json_data = requests.get(stac_item_url).json()

### Execute Resource Registration

In [14]:
register_url = f"{resource_registration_domain}/processes/register/execution"

payload = {
    "inputs": {
        "type": "dataset",
        "source": stac_item_url,
        "target": f"https://resource-catalogue.{platform_domain}/stac"
    }
}

response = requests.post(register_url, json=payload)

print(f"Dataset registration status code: {response.status_code}")
registration_result = response.json()
registration_result

Dataset registration status code: 200


{'id': 'registrar',
 'resource-and-data-catalogue-link': {'href': 'https://resource-catalogue.notebook-test.develop.eoepca.org/stac/collections/metadata:main/items/20201211_223832_CS2',
  'rel': 'item',
  'type': 'application/geo+json'}}

## Verify Registration Job Status

In [15]:
jobs_url = f"{resource_registration_domain}/jobs"
response = requests.get(jobs_url)
jobs = response.json()

print(f"Jobs listing status code: {response.status_code}")
latest_job = jobs["jobs"][0] if jobs.get("jobs") else {}
print(json.dumps(latest_job, indent=2))

Jobs listing status code: 200
{
  "type": "process",
  "processID": "register",
  "jobID": "11452c90-0ee7-11f0-8330-422a2051c4ad",
  "status": "failed",
  "message": "InvalidParameterValue: Error executing process: [\"$: 'type' is a required property\", \"$.source: {'content': {'stac_version': '1.1.0', 'stac_extensions': [], 'type': 'Feature', 'id': '20201211_223832_CS2', 'bbox': [172.91173669923782, 1.3438851951615003, 172.95469614953714, 1.3690476620161975], 'geometry': {'type': 'Polygon', 'coordinates': [[[172.91173669923782, 1.3438851951615003], [172.95469614953714, 1.3438851951615003], [172.95469614953714, 1.3690476620161975], [172.91173669923782, 1.3690476620161975], [172.91173669923782, 1.3438851951615003]]]}, 'properties': {'datetime': '2020-12-11T22:38:32.125000Z'}, 'collection': 'simple-collection', 'links': [{'rel': 'collection', 'href': './collection.json', 'type': 'application/json', 'title': 'Simple Example Collection'}, {'rel': 'root', 'href': './collection.json', 'type'

## Validate Registered Dataset in Resource Discovery

In [16]:
resource_discovery_domain = f'{os.environ.get("HTTP_SCHEME")}://resource-catalogue.{platform_domain}'
metadata_items_url = f"{resource_discovery_domain}/collections/metadata:main/items"

response = requests.get(metadata_items_url)
items = response.json().get("features", [])

item_ids = [item.get("id") for item in items]
expected_item_id = stac_item_json_data.get("id")

if expected_item_id in item_ids:
    print(f"✅ Dataset '{expected_item_id}' successfully registered and discoverable!")
else:
    print(f"❌ Dataset '{expected_item_id}' registration failed or item not discoverable.")

✅ Dataset '20201211_223832_CS2' successfully registered and discoverable!


## Detailed Inspection of `hello-world` Process

In [ ]:
hello_world_process_url = f"{resource_registration_domain}/processes/hello-world"
response = requests.get(hello_world_process_url)
print(json.dumps(response.json(), indent=2)[:500])

{
  "version": "0.2.0",
  "id": "hello-world",
  "title": "Hello World",
  "description": "An example process that takes a name as input, and echoes it back as output. Intended to demonstrate a simple process with a single literal input.",
  "jobControlOptions": [
    "sync-execute",
    "async-execute"
  ],
  "keywords": [
    "hello world",
    "example",
    "echo"
  ],
  "links": [
    {
      "type": "text/html",
      "rel": "about",
      "title": "information",
      "href": "https://exa


## Synchronous Execution of Hello-World Process

In [ ]:
hello_world_exec_url = f"{hello_world_process_url}/execution"
payload = {
    "inputs": {
        "name": "EOEPCA User",
        "message": "Testing synchronous execution"
    }
}
response = requests.post(hello_world_exec_url, json=payload)
print(response.json())

{'id': 'echo', 'value': 'Hello EOEPCA User! Testing synchronous execution'}


## Asynchronous Execution of Hello-World Process

In [ ]:
headers = {"Prefer": "respond-async"}
payload = {
    "inputs": {
        "name": "EOEPCA User",
        "message": "Testing asynchronous execution"
    }
}
response = requests.post(hello_world_exec_url, json=payload, headers=headers)
job_location = response.headers.get('Location')
print(f"Job created asynchronously. Location: {job_location}")

Job created asynchronously. Location: https://registration-api.notebook-test.develop.eoepca.org/jobs/34980a74-1530-11f0-aa49-422a2051c4ad


## Check Status of Async Job

In [ ]:
response = requests.get(job_location)
print(json.dumps(response.json(), indent=2))

{
  "type": "process",
  "processID": "hello-world",
  "jobID": "34980a74-1530-11f0-aa49-422a2051c4ad",
  "status": "successful",
  "message": "Job complete",
  "progress": 100,
  "parameters": null,
  "job_start_datetime": "2025-04-09T10:48:43.347927Z",
  "job_end_datetime": "2025-04-09T10:48:43.357530Z",
  "links": [
    {
      "href": "https://registration-api.notebook-test.develop.eoepca.org/jobs/34980a74-1530-11f0-aa49-422a2051c4ad/results?f=html",
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/results",
      "type": "text/html",
      "title": "Results of job as HTML"
    },
    {
      "href": "https://registration-api.notebook-test.develop.eoepca.org/jobs/34980a74-1530-11f0-aa49-422a2051c4ad/results?f=json",
      "rel": "http://www.opengis.net/def/rel/ogc/1.0/results",
      "type": "application/json",
      "title": "Results of job as JSON"
    }
  ]
}


## Retrieve Results from Async Job

In [ ]:
job_results_url = f"{job_location}/results"
response = requests.get(job_results_url, headers={"Accept": "application/json"})
print(response.json())

{'id': 'echo', 'value': 'Hello EOEPCA User! Testing asynchronous execution'}


## Additional Validation - Flowable API Check

In [17]:
flowable_domain = f'{os.environ.get("HTTP_SCHEME")}://registration-harvester-api.{platform_domain}'
flowable_docs_url = f"{flowable_domain}/flowable-rest/docs"
response = requests.get(flowable_docs_url)

print(f"Flowable Swagger UI status code: {response.status_code}")
response.url

Flowable Swagger UI status code: 200


'https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/docs/'

## List Flowable Deployed Processes

In [18]:
flowable_admin_user = os.environ.get("FLOWABLE_ADMIN_USER", "eoepca")
flowable_admin_password = os.environ.get("FLOWABLE_ADMIN_PASSWORD", "eoepca")

process_definitions_url = f"{flowable_domain}/flowable-rest/service/repository/process-definitions"

print(f"Flowable process definitions URL: {process_definitions_url}")

response = requests.get(process_definitions_url, auth=(flowable_admin_user, flowable_admin_password))

print(f"Flowable process definitions status code: {response.status_code}")
print(json.dumps(response.json(), indent=2))

Flowable process definitions URL: https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/process-definitions
Flowable process definitions status code: 200
{
  "data": [
    {
      "id": "createTimersProcess:1:641b2360-066c-11f0-9621-dac2883c12fb",
      "url": "https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/process-definitions/createTimersProcess:1:641b2360-066c-11f0-9621-dac2883c12fb",
      "key": "createTimersProcess",
      "version": 1,
      "name": "Create timers process",
      "description": "Test process to create a number of timers.",
      "tenantId": "",
      "deploymentId": "636540d0-066c-11f0-9621-dac2883c12fb",
      "deploymentUrl": "https://registration-harvester-api.notebook-test.develop.eoepca.org/flowable-rest/service/repository/deployments/636540d0-066c-11f0-9621-dac2883c12fb",
      "resource": "https://registration-harvester-api.notebook-test.develop.eoepca.org/

## Manually Trigger Landsat Data Harvesting Workflow

In [28]:
harvest_workflow_url = f"{flowable_domain}/flowable-rest/service/runtime/process-instances"
harvest_payload = {
    "processDefinitionKey": "createTimersProcess",
}
response = requests.post(harvest_workflow_url, json=harvest_payload, auth=(flowable_admin_user, flowable_admin_password))
print(response.json())

{'message': 'Internal server error', 'exception': "Unknown property used in expression: ${duration == 'long'}"}
